# Introduction to simple-C-elegans

*original available online: https://github.com/adammarblestone/simple-C-elegans/blob/master/SimulateConnectome.py*

This is essentially a single python script that simulates the nervous system of the C. elegans; with many caveats known to the authors.  The goal is purposeful simplicity to be able to demonstrate the concepts surrounding simulating a nervous system with a reasonable amount of biological detail.  This walkthrough will explain the simple-C-elegans code and how it works.  

A more robust approach is being taken in the OpenWorm project that involves many pieces of infrastructure that are more sophisticated, but also less simple.  Therefore, in addition to explaining how this code works, the larger OpenWorm infrastructure that is responsible for various steps will also be explained along the way.

This notebook resides within the repo at https://github.com/openworm/simple-C-elegans/ 

Original code author: Adam Marblestone

Notebook author: Stephen Larson

Comments at the top say:
    
>Simulates the dynamics of part or all of the C. elegans neural network.

>This code is *deliberately simple* in order to help the user gain intuition for the qualitative properties of 
the network dynamics.

>Key references include:

> 1) Wicks, Stephen R., Chris J. Roehrig, and Catharine H. Rankin. "A dynamic network simulation of the nematode tap withdrawal circuit: predictions concerning synaptic function using behavioral criteria." The Journal of neuroscience 16.12 (1996): 4017-4031. http://www.ncbi.nlm.nih.gov/pubmed/8656295

> 2) Kunert, James, Eli Shlizerman, and J. Nathan Kutz. "Low-dimensional functionality of complex network dynamics: Neurosensory integration in the Caenorhabditis elegans connectome." Physical Review E 89.5 (2014): 052805. http://arxiv.org/abs/1310.6689

>3) http://www.openworm.org/ and the associated resources

First, some practicalities.  The [sci-py libraries](https://www.scipy.org/install.html) are needed to make this run.  One particularly nice setup can be found here: https://github.com/jupyter/docker-stacks/tree/master/scipy-notebook with the setup via Docker.

In addition, installing neuroml will be helpful:

In [9]:
!sudo apt-get update
!sudo apt-get install -y libxml2-dev libxslt-dev
!pip2 install libneuroml

Hit http://security.debian.org jessie/updates InRelease
Get:1 http://security.debian.org jessie/updates/main amd64 Packages [384 kB]
Ign http://httpredir.debian.org jessie InRelease
Hit http://httpredir.debian.org jessie-updates InRelease
Hit http://httpredir.debian.org jessie Release.gpg
Get:2 http://httpredir.debian.org jessie-updates/main amd64 Packages [17.6 kB]
Hit http://httpredir.debian.org jessie Release
Get:3 http://httpredir.debian.org jessie/main amd64 Packages [9,032 kB]
Fetched 9,433 kB in 7s (1,197 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libxslt1-dev' instead of 'libxslt-dev'
libxml2-dev is already the newest version.
libxslt1-dev is already the newest version.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
  Using cached libNeuroML-0.2.18.tar.gz
  Using cached lxml-3.6.1.tar.gz
  Running setup.py bdist_wheel for libneuroml ... - \ done
  Stored

After these installs, now the imports from the script should work:

In [10]:
import neuroml
import neuroml.loaders as loaders
import neuroml.writers as writers
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
from scipy.integrate import odeint
import numpy as np
import random
import matplotlib.animation as animation
import time
import pickle
import os

/opt/conda/envs/python2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Seems like a switch to archive stuff or not.  Not sure why this isn't always true (probably takes more time).

In [12]:
usePickle = False

## Connectome parameters.

In [13]:
connectome_spreadsheet = "OpenWorm/connectome.csv"

# all the neurons
all_the_cell_names = ["ADAL","ADAR","ADEL","ADER","ADFL","ADFR","ADLL","ADLR","AFDL","AFDR","AIAL","AIAR","AIBL","AIBR","AIML","AIMR","AINL","AINR","AIYL","AIYR","AIZL","AIZR","ALA","ALML","ALMR","ALNL","ALNR","AQR","AS1","AS2","AS3","AS4","AS5","AS6","AS7","AS8","AS9","AS10","AS11","ASEL","ASER","ASGL","ASGR","ASHL","ASHR","ASIL","ASIR","ASJL","ASJR","ASKL","ASKR","AUAL","AUAR","AVAL","AVAR","AVBL","AVBR","AVDL","AVDR","AVEL","AVER","AVFL","AVFR","AVG","AVHL","AVHR","AVJL","AVJR","AVKL","AVKR","AVL","AVM","AWAL","AWAR","AWBL","AWBR","AWCL","AWCR","BAGL","BAGR","BDUL","BDUR","CEPDL","CEPDR","CEPVL","CEPVR","DA1","DA2","DA3","DA4","DA5","DA6","DA7","DA8","DA9","DB1","DB2","DB3","DB4","DB5","DB6","DB7","DD1","DD2","DD3","DD4","DD5","DD6","DVA","DVB","DVC","FLPL","FLPR","HSNL","HSNR","I1L","I1R","I2L","I2R","I3","I4","I5","I6","IL1DL","IL1DR","IL1L","IL1R","IL1VL","IL1VR","IL2DL","IL2DR","IL2L","IL2R","IL2VL","IL2VR","LUAL","LUAR","M1","M2L","M2R","M3L","M3R","M4","M5","MCL","MCR","MI","NSML","NSMR","OLLL","OLLR","OLQDL","OLQDR","OLQVL","OLQVR","PDA","PDB","PDEL","PDER","PHAL","PHAR","PHBL","PHBR","PHCL","PHCR","PLML","PLMR","PLNL","PLNR","PQR","PVCL","PVCR","PVDL","PVDR","PVM","PVNL","PVNR","PVPL","PVPR","PVQL","PVQR","PVR","PVT","PVWL","PVWR","RIAL","RIAR","RIBL","RIBR","RICL","RICR","RID","RIFL","RIFR","RIGL","RIGR","RIH","RIML","RIMR","RIPL","RIPR","RIR","RIS","RIVL","RIVR","RMDDL","RMDDR","RMDL","RMDR","RMDVL","RMDVR","RMED","RMEL","RMER","RMEV","RMFL","RMFR","RMGL","RMGR","RMHL","RMHR","SAADL","SAADR","SAAVL","SAAVR","SABD","SABVL","SABVR","SDQL","SDQR","SIADL","SIADR","SIAVL","SIAVR","SIBDL","SIBDR","SIBVL","SIBVR","SMBDL","SMBDR","SMBVL","SMBVR","SMDDL","SMDDR","SMDVL","SMDVR","URADL","URADR","URAVL","URAVR","URBL","URBR","URXL","URXR","URYDL","URYDR","URYVL","URYVR","VA1","VA2","VA3","VA4","VA5","VA6","VA7","VA8","VA9","VA10","VA11","VA12","VB1","VB2","VB3","VB4","VB5","VB6","VB7","VB8","VB9","VB10","VB11", "VC1", "VC2", "VC3","VC4","VC5", "VD1", "VD2", "VD3", "VD4", "VD5", "VD6", "VD7", "VD8", "VD9", "VD10", "VD11", "VD12", "VD13"] 

# just the core sensory + interneuron circuit from Wicks + forward motor neurons: DA, VA and AS would need to be added for backwards motion
# all_the_cell_names = ["AVM", "AVDL","AVDR", "AVAL","AVAR","AVBL","AVBR","ALML","ALMR","PLML","PLMR","PVCL","PVCR","PVDL","PVDR","DVA","DB1","DB2","DB3","DB4","DB5","DB6","DB7","DD1","DD2","DD3","DD4","DD5","DD6","VB1","VB2","VB3","VB4","VB5","VB6","VB7","VB8","VB9", "VB10","VB11", "VD1", "VD2", "VD3", "VD4", "VD5", "VD6", "VD7", "VD8", "VD9", "VD10", "VD11", "VD12", "VD13"] 

# just the core sensory + interneuron circuit from Wicks
# all_the_cell_names = ["AVM", "AVDL","AVDR", "AVAL","AVAR","AVBL","AVBR","ALML","ALMR","PLML","PLMR","PVCL","PVCR","PVDL","PVDR","DVA"] 

# core sensory + interneuron circuit from Wicks + LUAL and LUAR which the Hiroshima group uses: http://www.bsys.hiroshima-u.ac.jp/pub/pdf/J/J_153.pdf
# all_the_cell_names = ["AVM", "AVDL","AVDR", "AVAL","AVAR","AVBL","AVBR","ALML","ALMR","PLML","PLMR","PVCL","PVCR","PVDL","PVDR","DVA","LUAL","LUAR"] 

# core sensory + interneuron  circuit + LUAL and LUAR + forward motor neurons
#all_the_cell_names = ["AVM", "AVDL","AVDR", "AVAL","AVAR","AVBL","AVBR","ALML","ALMR","PLML","PLMR","PVCL","PVCR","PVDL","PVDR","DVA","LUAL","LUAR", "DB1","DB2","DB3","DB4","DB5","DB6","DB7","DD1","DD2","DD3","DD4","DD5","DD6","VB1","VB2","VB3","VB4","VB5","VB6","VB7","VB8","VB9", "VB10","VB11", "VD1", "VD2", "VD3", "VD4", "VD5", "VD6", "VD7", "VD8", "VD9", "VD10", "VD11", "VD12", "VD13"] 

# core sensory + interneuron  circuit from Wicks + LUAL and LUAR + forward motor neurons + backwards motor neurons
# all_the_cell_names = ["AVM", "AVDL","AVDR", "AVAL","AVAR","AVBL","AVBR","ALML","ALMR","PLML","PLMR","PVCL","PVCR","PVDL","PVDR","DVA","LUAL","LUAR", "DB1","DB2","DB3","DB4","DB5","DB6","DB7","DD1","DD2","DD3","DD4","DD5","DD6","VB1","VB2","VB3","VB4","VB5","VB6","VB7","VB8","VB9", "VB10","VB11", "VD1", "VD2", "VD3", "VD4", "VD5", "VD6", "VD7", "VD8", "VD9", "VD10", "VD11", "VD12", "VD13", "DA1","DA2","DA3","DA4","DA5","DA6","DA7","DA8","DA9","VA1","VA2","VA3","VA4","VA5","VA6","VA7","VA8","VA9","VA10","VA11","VA12", "AS1","AS2","AS3","AS4","AS5","AS6","AS7","AS8","AS9","AS10","AS11"]

l = len(all_the_cell_names)
indices = range(l)

coords = [] # list of 3D coordinates of all the neurons
cells_connections = {} # synaptic
cells_gaps = {} # gap junctional

connection_nums = {} # synaptic connection nums dict
gap_nums = {} # gap junction connection nums dict

GABA_adjacency = {} # synaptic connection nums dict specific to GABA
Chol_adjacency = {} # synaptic connection nums dict specific to Acetylcholine
Glut_adjacency = {} # synaptic connection nums dict specific to Glutamate
Other_adjacency = {} # synaptic connection nums dict specific to Other
GABA_Chol_Glut_adjacency = {} # synaptic connection nums dict including everything except "Other"

GABA_expressing = [] # list of pre-synaptic cells expressing GABA
Cholinergic = [] # list of pre-synaptic cells expressing Acetylcholine
Glutamatergic = [] # list of pre-synaptic cells expressing Glutamate
labeled_neuron = "PLML" # give a particular neuron a different color
coords_of_labeled_neuron = (None, None, None)

'''Simulation parameters'''
# params of the simulation
I_ext_magnitude = 0.5 * 1e-12 # external current in units of Amperes
stim_dur = 0.5
stim_start = 0.5
start_time = 0.0
end_time = 1.5
I_ext_mask = [1.0 if all_the_cell_names[i] in ["PLML", "PLMR"] else 0.0 for i in indices] # which neurons get external current input

'''Dynamical parameters.'''
enhanced_synaptic_steepness = 8.0 # Important -- set this to > 1 to increase the nonlinearity: it appears to be effectively set to 1.0, however, in the Kunert and Wicks papers
K = -4.39 * enhanced_synaptic_steepness # pre-factor in the exponential synapse activation
G_c = 100.0 * 1e-12 # cell membrane conductance (S)
C = 10.0 * 1e-12 # cell membrane capacitance (F)
g_syn = 10.0 * 1e-12 # synaptic conductance (S)
g_gap = 5.0 * 1e-12 # gap junctional conductance (S)
G_c_over_C = G_c/C
g_syn_over_C = g_syn/C
g_gap_over_C = g_gap/C
E_c = [-35.0 * 1e-3 for i in indices] # -35 mV leakage potentials
V_range = 35.0 * 1e-3 # 35 mV range
relative_GABA = 0.5 # relative strength of GABA compared to Glut and Chol synapses

## Main loop

In [ ]:
def main():          
    # get each cell's soma coords
    importData()
    
    # read connectivity data
    readSynapticConnectome()
    readGapConnectome()
    
    # plot the connectome in 3D
    plotConnectomeIn3D()
    
    # plot adjacency matrices
    buildAdjacency()
    plotAdjacency()
    
    # calculate IV curves: this takes a long time so it is usually commented
    # plotIVcurves()
    
    # integrate the ODEs
    soln = integrateODEs(1)
    
    # plot the states of the synaptic parameters
    fig = plt.figure()
    plt.xlabel("Voltage (V)")
    plt.ylabel("Relative pre-synaptic activation || Time (au)")
    plt.title("Comparison of voltages to synaptic activations")
    V_eq = findEquilibriumPotentials()
    V_vals_to_plot = np.linspace(-50 * 1e-3, 0.0 * 1e-3, num = 100, endpoint = True)
    for k in indices:
        plt.plot(V_vals_to_plot, [1.0/(1+np.exp(K*(V_vals_to_plot[i] - V_eq[k])/V_range)) for i in range(len(V_vals_to_plot))], color='r')
        
    for i in indices:
        plt.plot([float(p) for p in soln[:,i]], [1*float(p)/len(soln[:,i]) for p in range(len(soln[:,i]))], color = 'k')
    
    # calculate Jacobian
    calculateJacobian(soln)
    
    # run SVD
    if len(all_the_cell_names) > 18: # this IF statement just checks to be sure there are motor neurons in the simulation to run SVD on!
        runSVD(soln)
    
    # set up plots for the animation
    #setupAnimation(soln)
    
    # show the plots
    plt.show()